<img src='pics/otus.png'>

In [2]:
from __future__ import print_function
from collections import defaultdict
import pandas as pd
import numpy as np
import matplotlib
import pandas_profiling
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sns

%matplotlib inline
plt.rcParams["figure.figsize"] = (15, 8)
pd.options.display.float_format = '{:.2f}'.format

In [3]:
fn = 'orders.csv'

In [4]:
!head $fn

DISCOUNT_VALUE,NAME,PRICE,QUANTITY,date_insert,discount,itemID,orderID,ordered_item_count,price,user_id
23.55%,Хроники Амбера. Том 2,357.0,1,2017-03-19 22:25:20,110.0,169108,602051,1,527.0,911531
4.07%,Хроники Амбера. Том 1,448.0,1,2017-03-19 22:19:58,19.0,1394744,602047,1,618.0,911531
4.17%,Пищеводитель,414.0,1,2017-03-19 20:58:00,65.0,191276,602017,4,1503.0,911587
4.17%,"Аудиокн. Мясников. &quot;Ржавчина&quot;: что делать, чтобы сердце не болело",253.0,1,2017-03-19 20:58:00,65.0,157945,602017,4,1503.0,911587
4.09%,Инфекции. Как защитить себя и своего ребенка,422.0,1,2017-03-19 20:58:00,65.0,205173,602017,4,1503.0,911587
4.17%,О самом главном с доктором Мясниковым,414.0,1,2017-03-19 20:58:00,65.0,189415,602017,4,1503.0,911587
4.12%,Пеппи Длинныйчулок собирается в путь,489.0,1,2017-03-19 20:53:51,42.0,168183,602015,2,1238.0,201724
4.12%,Пеппи Длинныйчулок в стране Веселии,489.0,1,2017-03-19 20:53:51,42.0,186112,602015,2,1238.0,201724
4.09%,Серебряный шар. Драма за сценой,586.0,1,2017-0

# 1. Провести EDA
Просмотреть структуру данных, значения каждого признака, поискать особенности в признаках и в данных вцелом

In [45]:
df = pd.read_csv('orders.csv')

In [46]:
df.shape

(557818, 11)

In [72]:
df.head(10)

,DISCOUNT_VALUE,NAME,PRICE,QUANTITY,date_insert,discount,itemID,orderID,ordered_item_count,price,user_id
0,23.55%,Хроники Амбера. Том 2,357.00,1,2017-03-19 22:25:20,110.00,169108,602051,1,527.00,911531
1,4.07%,Хроники Амбера. Том 1,448.00,1,2017-03-19 22:19:58,19.00,1394744,602047,1,618.00,911531
2,4.17%,Пищеводитель,414.00,1,2017-03-19 20:58:00,65.00,191276,602017,4,1503.00,911587
3,4.17%,Аудиокн. Мясников. &quot;Ржавчина&quot;: что д...,253.00,1,2017-03-19 20:58:00,65.00,157945,602017,4,1503.00,911587
4,4.09%,Инфекции. Как защитить себя и своего ребенка,422.00,1,2017-03-19 20:58:00,65.00,205173,602017,4,1503.00,911587
5,4.17%,О самом главном с доктором Мясниковым,414.00,1,2017-03-19 20:58:00,65.00,189415,602017,4,1503.00,911587
6,4.12%,Пеппи Длинныйчулок собирается в путь,489.00,1,2017-03-19 20:53:51,42.00,168183,602015,2,1238.00,201724
7,4.12%,Пеппи Длинныйчулок в стране Веселии,489.00,1,2017-03-19 20:53:51,42.00,186112,602015,2,1238.00,201724
8,4.09%,Серебряный шар. Драма за сценой,586.00,1,2017-03-19 20:28:54,25.00,181388,602005,1,586.00,77858
9,4.49%,Тимур и его команда,170.00,1,2017-03-19 20:23:24,15.00,149381,602004,2,562.00,355403


In [48]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 557818 entries, 0 to 557817
Data columns (total 11 columns):
DISCOUNT_VALUE        553777 non-null object
NAME                  557811 non-null object
PRICE                 557818 non-null float64
QUANTITY              557818 non-null int64
date_insert           557818 non-null object
discount              557818 non-null float64
itemID                557818 non-null int64
orderID               557818 non-null int64
ordered_item_count    557818 non-null int64
price                 557818 non-null float64
user_id               557818 non-null int64
dtypes: float64(3), int64(5), object(3)
memory usage: 46.8+ MB


Как видно, количество признаков не так уж велико и кроме DISCOUNT_VALUE и NAME они заполненны полностью.

#### Попробуем составить описание признаков руководствуясь здравым смыслом:

<ul>
<li>DISCOUNT_VALUE - процент скидки, категориальный признак, который легко трансформируется в количественный</li>
<li>NAME - имя товара, категориальный признак</li>
<li>PRICE - цена товара, количественный признак</li>
<li>QUANTITY - количество товаров в заказе, количественный признак</li>
<li>date_insert - дата заказа или формирования заказа, порядковый признак</li>
<li>discount - скидка, количественный признак</li>
<li>itemID - идентификатор товара, порядковый признак</li>
<li>orderID - идентификатор заказа, порядковый признак</li>
<li>ordered_item_count - количественный признак</li>
<li>price - количественный признак</li>
<li>user_id - идентификатор пользователя, порядковый признак</li>
</ul>

#### Проверим на дубликаты и удалим если такие имеются:

In [49]:
df.duplicated().sum()

10

In [50]:
df.drop_duplicates(inplace = True)

In [51]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 557808 entries, 0 to 557817
Data columns (total 11 columns):
DISCOUNT_VALUE        553771 non-null object
NAME                  557801 non-null object
PRICE                 557808 non-null float64
QUANTITY              557808 non-null int64
date_insert           557808 non-null object
discount              557808 non-null float64
itemID                557808 non-null int64
orderID               557808 non-null int64
ordered_item_count    557808 non-null int64
price                 557808 non-null float64
user_id               557808 non-null int64
dtypes: float64(3), int64(5), object(3)
memory usage: 51.1+ MB


#### Теперь надо разобраться как считается сумма заказа и скидка:

Для примера возьмем заказ orderID - 602017

In [75]:
orders = df[df['orderID'] == 602017]
orders.head()

,DISCOUNT_VALUE,NAME,PRICE,QUANTITY,date_insert,discount,itemID,orderID,ordered_item_count,price,user_id
2,4.17%,Пищеводитель,414.00,1,2017-03-19 20:58:00,65.00,191276,602017,4,1503.00,911587
3,4.17%,Аудиокн. Мясников. &quot;Ржавчина&quot;: что д...,253.00,1,2017-03-19 20:58:00,65.00,157945,602017,4,1503.00,911587
4,4.09%,Инфекции. Как защитить себя и своего ребенка,422.00,1,2017-03-19 20:58:00,65.00,205173,602017,4,1503.00,911587
5,4.17%,О самом главном с доктором Мясниковым,414.00,1,2017-03-19 20:58:00,65.00,189415,602017,4,1503.00,911587


In [ ]:
print('Sum of PRICE:', orders['PRICE'].sum())
print('price:', orders['price'].iloc[0])
orders['DISCOUNT_VALUE']

In [60]:
print(orders['price'].iloc[0] / (100 - 4.17) * 100)
print(orders['price'].iloc[0] / (100 - 4.09) * 100)

1568.40237921
1567.09415077


In [67]:
price_without_discount = (
    (orders['price'].iloc[0] / (100 - 4.17) * 100) * 3 + orders['price'].iloc[0] / (100 - 4.09) * 100
) / 4
price_without_discount

1568.0753221014786

In [70]:
print('discount:', price_without_discount - orders.iloc[0]['price'])

discount 65.0753221015


Получается, что скидка формируется от цены заказа и "немного" не соответсвует скидке, если рассчитывать ее по DISCOUNT_VALUE.

# 2. Спланировать эксперимент 

Задача:
* сегментация товаров + сегментация пользователей (по товарам, по поведению, скидкам и пр.)

Дополнительнительные задачи (опционально, подумайте о них, могут появиться интересные идеи и находки):
* поиск товаров-заменителей и товаров, дополняющих друг друга
* поиск выбросов среди товаров
* поиск выбросов среди заказов

Эксперимент - составить план, по которому будет проделана исследовательская работа. Примерные пункты плана:
* Составить первичный список признаков, необходимых для решения задачи
* Сгенерировать список новых признаков, которые можно получить из данных
* Применить понижение размерности для визуализации данных
* Применить кластеризацию, визуализировать
* Описание кластера
* Любые другие пункты

# 3. Выполнить подготовку данных для построения модели

Выполнить необходимые преобразования данных

# 4. Провести построение модели 

# 5. Проанализировать результат
получилось или нет, почему, что можно улучшить, можно ли интерпретировать результат

# 6. Повторить при необходимости 

# Подсказки

* Проводить EDA и работу с признаками на всем датасете
* Для построения модели отсеивать данные тем или иным способом (обосновать выбор) - уменьшить объем данных для анализа
* Оформлять для сдачи задания все эксперименты (в том числе неудачные). Так мы сможем увидеть процесс работы над заданием
* Не тратить 90% времени на первый этап. Планировать время так, чтобы успеть поработать над моделью (в равной степени над каждым пунктом).